In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import backend as K
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
# path to the model weights files.
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/mnt/safe01/data/public/isic/train'
validation_data_dir = '/mnt/safe01/data/public/isic/test'
nb_train_samples = 4048
nb_validation_samples = 480
epochs = 50
batch_size = 16

In [3]:
# build the VGG16 network
# set channels first notation
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
print(input_shape)
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
print('Model loaded.')

(150, 150, 3)
Model loaded.


In [4]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [5]:
top_model.load_weights(top_model_weights_path)
input_tensor = Input(shape=(input_shape))
base_model = VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
# add the model on top of the convolutional base
model = Model(input= base_model.input, output= top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
metrics=['accuracy'])

/home/mydang/food101/myfood101/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  


In [6]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
class_mode='binary')

Found 4053 images belonging to 2 classes.
Found 482 images belonging to 2 classes.


In [7]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
nb_val_samples=nb_validation_samples)

/home/mydang/food101/myfood101/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/mydang/food101/myfood101/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=253, epochs=50, validation_steps=480)`
  import sys


Epoch 1/50
253/253 [==============================] - 2683s - loss: 0.9419 - acc: 0.7125 - val_loss: 0.8753 - val_acc: 0.7116
Epoch 2/50
253/253 [==============================] - 2690s - loss: 0.9056 - acc: 0.7114 - val_loss: 0.8694 - val_acc: 0.7114
Epoch 3/50
253/253 [==============================] - 2415s - loss: 0.9667 - acc: 0.7016 - val_loss: 0.8704 - val_acc: 0.7112
Epoch 4/50
253/253 [==============================] - 2417s - loss: 0.9360 - acc: 0.7099 - val_loss: 0.8742 - val_acc: 0.7123
Epoch 5/50
253/253 [==============================] - 2402s - loss: 0.9706 - acc: 0.7020 - val_loss: 0.8713 - val_acc: 0.7107
Epoch 6/50
253/253 [==============================] - 2420s - loss: 0.9851 - acc: 0.6987 - val_loss: 0.8727 - val_acc: 0.7120
Epoch 7/50
253/253 [==============================] - 2417s - loss: 0.9027 - acc: 0.7123 - val_loss: 0.8703 - val_acc: 0.7112
Epoch 8/50
253/253 [==============================] - 2409s - loss: 0.9694 - acc: 0.7057 - val_loss: 0.8681 - val_acc:

KeyboardInterrupt: 